<a href="https://colab.research.google.com/github/tomonari-masada/course2021-nlp/blob/main/10_machine_translation_with_transformer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# トランスフォーマを使った機械翻訳

* データセットとして[Multi30k](http://www.statmt.org/wmt16/multimodal-task.html#task1)を使う。 
* ここでは、ドイツ語から英語への翻訳を行なう。



## 準備

* 使えるGPUを確認

In [2]:
!nvidia-smi

Sun Nov 14 10:49:49 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P8    27W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

* spaCy関係のインストール

In [3]:
!pip install -U spacy
!python -m spacy download en_core_web_sm
!python -m spacy download de_core_news_sm

     |████████████████████████████████| 6.0 MB 4.1 MB/s 
     |████████████████████████████████| 628 kB 49.0 MB/s 
     |████████████████████████████████| 10.1 MB 35.4 MB/s 
     |████████████████████████████████| 181 kB 50.2 MB/s 
     |████████████████████████████████| 451 kB 51.3 MB/s 
     |████████████████████████████████| 42 kB 1.3 MB/s 
  Attempting uninstall: catalogue
    Found existing installation: catalogue 1.0.0
    Uninstalling catalogue-1.0.0:
      Successfully uninstalled catalogue-1.0.0
  Attempting uninstall: srsly
    Found existing installation: srsly 1.0.5
    Uninstalling srsly-1.0.5:
      Successfully uninstalled srsly-1.0.5
  Attempting uninstall: thinc
    Found existing installation: thinc 7.4.0
    Uninstalling thinc-7.4.0:
      Successfully uninstalled thinc-7.4.0
  Attempting uninstall: spacy
    Found existing installation: spacy 2.2.4
    Uninstalling spacy-2.2.4:
      Successfully uninstalled spacy-2.2.4
     |████████████████████████████████| 13.9 M

## データの前処理
* [`torchtext`](https://pytorch.org/text/stable/) を使うと便利。

### 翻訳元の言語のトークナイザと翻訳先の言語のトークナイザを作る

In [4]:
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
from torchtext.datasets import Multi30k
from typing import Iterable, List

SRC_LANGUAGE = 'de'
TGT_LANGUAGE = 'en'

# Place-holder
token_transform = {}

token_transform[SRC_LANGUAGE] = get_tokenizer('spacy', language='de_core_news_sm')
token_transform[TGT_LANGUAGE] = get_tokenizer('spacy', language='en_core_web_sm')

### トークンのリストを作るヘルパー関数の定義

In [5]:
def yield_tokens(data_iter: Iterable, language: str) -> List[str]:
  language_index = {SRC_LANGUAGE: 0, TGT_LANGUAGE: 1}

  for data_sample in data_iter:
    yield token_transform[language](data_sample[language_index[language]])

### 特殊なシンボルとそのインデックスを定義
* インデックスとリスト内での順番が合っていることを確認する。

In [6]:
UNK_IDX, PAD_IDX, BOS_IDX, EOS_IDX = 0, 1, 2, 3
special_symbols = ['<unk>', '<pad>', '<bos>', '<eos>']

### 両言語のデータから語彙集合を作る

In [7]:
# Place-holder 
vocab_transform = {}

for ln in [SRC_LANGUAGE, TGT_LANGUAGE]:
  # 訓練データのイテレータ
  train_iter = Multi30k(split='train', language_pair=(SRC_LANGUAGE, TGT_LANGUAGE))
  # torchtextのVocabオブジェクトを作る 
  vocab_transform[ln] = build_vocab_from_iterator(yield_tokens(train_iter, ln),
                                                  min_freq=1,
                                                  specials=special_symbols,
                                                  special_first=True)

# UNK_IDXをデフォルトのインデックスとして設定する（語彙集合に見つからなかった単語のインデックス）
# これを実行しないと実行時にエラーが出る
for ln in [SRC_LANGUAGE, TGT_LANGUAGE]:
  vocab_transform[ln].set_default_index(UNK_IDX)

100%|██████████| 1.21M/1.21M [00:02<00:00, 482kB/s]


## Seq2Seqネットワークをトランスフォーマを使って実装する

* ここでは、論文[`“Attention is all you
need”`](https://papers.nips.cc/paper/2017/file/3f5ee243547dee91fbd053c1c4a845aa-Paper.pdf)で提案されたモデルを実装する。
* ネットワークは三つの部分から成る。
 * 埋め込みレイヤ（単語インデックスを埋め込みベクトルに変換する）
 * トランスフォーマモデル
 * 全結合層（各トークンに対して、規格化されていない確率の値を出力する）

### 準備

In [8]:
from torch import Tensor
import torch
import torch.nn as nn
from torch.nn import Transformer
import math
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

### トークンの位置をエンコードするモジュールの定義
* 注意機構がトークンの出現位置（先頭から何番目か）を反映しない計算であることに注意。
* そのため、何番目のトークンであるかを表すベクトルをトークン埋め込みに加算する。
* `torch.nn.Module.register_buffer`については、[ここ](https://pytorch.org/docs/stable/generated/torch.nn.Module.html?highlight=register_buffer#torch.nn.Module.register_buffer)を参照。
* 埋め込み層の出力にドロップアウトが使われている点に注意。

In [13]:
class PositionalEncoding(nn.Module):
  def __init__(self,
               emb_size: int,
               dropout: float,
               maxlen: int = 5000):
    super(PositionalEncoding, self).__init__()
    den = torch.exp(- torch.arange(0, emb_size, 2)* math.log(10000) / emb_size)
    pos = torch.arange(0, maxlen).reshape(maxlen, 1)
    pos_embedding = torch.zeros((maxlen, emb_size))
    pos_embedding[:, 0::2] = torch.sin(pos * den)
    pos_embedding[:, 1::2] = torch.cos(pos * den)
    pos_embedding = pos_embedding.unsqueeze(-2)

    self.dropout = nn.Dropout(dropout)
    self.register_buffer('pos_embedding', pos_embedding)

  def forward(self, token_embedding: Tensor):
    # token_embedding.size(0)はミニバッチ内部で最長のトークン列の長さ
    return self.dropout(token_embedding + self.pos_embedding[:token_embedding.size(0), :])

### トークンのインデックス列を埋め込みベクトル列へ変換するモジュールの定義
* `torch.nn.Embedding`が各単語の埋め込みベクトルを表す。
* この`torch.nn.Embedding`も含めてtrainingする。

In [9]:
class TokenEmbedding(nn.Module):
  def __init__(self, vocab_size: int, emb_size):
    super(TokenEmbedding, self).__init__()
    self.embedding = nn.Embedding(vocab_size, emb_size)
    self.emb_size = emb_size

  def forward(self, tokens: Tensor):
    return self.embedding(tokens.long()) * math.sqrt(self.emb_size)

### トランスフォーマを使ったseq2seqネットワークの定義
* これが本体。

In [10]:
class Seq2SeqTransformer(nn.Module):
  def __init__(self,
               num_encoder_layers: int,
               num_decoder_layers: int,
               emb_size: int,
               nhead: int,
               src_vocab_size: int,
               tgt_vocab_size: int,
               dim_feedforward: int = 512,
               dropout: float = 0.1):
    super(Seq2SeqTransformer, self).__init__()
    self.transformer = Transformer(d_model=emb_size,
                                   nhead=nhead,
                                   num_encoder_layers=num_encoder_layers,
                                   num_decoder_layers=num_decoder_layers,
                                   dim_feedforward=dim_feedforward,
                                   dropout=dropout)
    self.generator = nn.Linear(emb_size, tgt_vocab_size)
    self.src_tok_emb = TokenEmbedding(src_vocab_size, emb_size)
    self.tgt_tok_emb = TokenEmbedding(tgt_vocab_size, emb_size)
    self.positional_encoding = PositionalEncoding(
        emb_size, dropout=dropout)

  def forward(self,
              src: Tensor,
              trg: Tensor,
              src_mask: Tensor,
              tgt_mask: Tensor,
              src_padding_mask: Tensor,
              tgt_padding_mask: Tensor,
              memory_key_padding_mask: Tensor):
    src_emb = self.positional_encoding(self.src_tok_emb(src))
    tgt_emb = self.positional_encoding(self.tgt_tok_emb(trg))
    outs = self.transformer(src_emb, tgt_emb, src_mask, tgt_mask, None,
                            src_padding_mask, tgt_padding_mask, memory_key_padding_mask)
    return self.generator(outs)

  def encode(self, src: Tensor, src_mask: Tensor):
    return self.transformer.encoder(self.positional_encoding(self.src_tok_emb(src)),
                                    src_mask)

  def decode(self, tgt: Tensor, memory: Tensor, tgt_mask: Tensor):
    return self.transformer.decoder(self.positional_encoding(self.tgt_tok_emb(tgt)),
                                    memory,
                                    tgt_mask)

### 未来のトークンを見ないようにするためのマスクを作るヘルパ関数
* これは生成側（翻訳先）だけに関係することに注意。

In [11]:
def generate_square_subsequent_mask(sz):
  mask = (torch.triu(torch.ones((sz, sz), device=DEVICE)) == 1).transpose(0, 1)
  mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
  return mask

### 翻訳元と翻訳先の文ペアに対してマスクを作るヘルパ関数
* パディング用トークンを見ないようにするためのマスクも含めて返す。

In [20]:
def create_mask(src, tgt):
  src_seq_len = src.shape[0]
  tgt_seq_len = tgt.shape[0]

  tgt_mask = generate_square_subsequent_mask(tgt_seq_len)
  src_mask = torch.zeros((src_seq_len, src_seq_len),device=DEVICE).type(torch.bool)

  src_padding_mask = (src == PAD_IDX).transpose(0, 1)
  tgt_padding_mask = (tgt == PAD_IDX).transpose(0, 1)
  return src_mask, tgt_mask, src_padding_mask, tgt_padding_mask

## 学習のための準備
* モデルパラメータの初期化方法に注意。
* 損失関数はクロスエントロピー。
* 最適化アルゴリズムはAdam。

In [14]:
torch.manual_seed(0)

SRC_VOCAB_SIZE = len(vocab_transform[SRC_LANGUAGE])
TGT_VOCAB_SIZE = len(vocab_transform[TGT_LANGUAGE])
EMB_SIZE = 512
NHEAD = 8
FFN_HID_DIM = 512
BATCH_SIZE = 128
NUM_ENCODER_LAYERS = 3
NUM_DECODER_LAYERS = 3

transformer = Seq2SeqTransformer(NUM_ENCODER_LAYERS, NUM_DECODER_LAYERS, EMB_SIZE,
                                 NHEAD, SRC_VOCAB_SIZE, TGT_VOCAB_SIZE, FFN_HID_DIM)

for p in transformer.parameters():
  if p.dim() > 1:
    nn.init.xavier_uniform_(p)

transformer = transformer.to(DEVICE)

loss_fn = torch.nn.CrossEntropyLoss(ignore_index=PAD_IDX)

optimizer = torch.optim.Adam(transformer.parameters(), lr=0.0001, betas=(0.9, 0.98), eps=1e-9)

## Collationのための関数
* collationとは、バラバラの文字列を前処理し、テンソルとしてのミニバッチへまとめあげる操作。

In [15]:
from torch.nn.utils.rnn import pad_sequence

# 所定の変換用の関数を使ってトークン列を変換する関数を返す関数
def sequential_transforms(*transforms):
  def func(txt_input):
    for transform in transforms:
      txt_input = transform(txt_input)
    return txt_input
  return func

# BOS/EOSトークンを先頭/末尾に追加する関数
def tensor_transform(token_ids: List[int]):
  return torch.cat((torch.tensor([BOS_IDX]),
                    torch.tensor(token_ids), 
                    torch.tensor([EOS_IDX])))

# src and tgt language text transforms to convert raw strings into tensors indices
text_transform = {}
for ln in [SRC_LANGUAGE, TGT_LANGUAGE]:
  text_transform[ln] = sequential_transforms(token_transform[ln], #トークナイゼーション
                                             vocab_transform[ln], #ベクトル列へ変換
                                             tensor_transform) #BOS/EOSを追加しテンソル化


# ばらばらの文字列サンプルをミニバッチへcollateする関数
# （DataLoaderのインスタンスを作るときにこの関数を指定する）
def collate_fn(batch):
  src_batch, tgt_batch = [], []
  for src_sample, tgt_sample in batch:
    src_batch.append(text_transform[SRC_LANGUAGE](src_sample.rstrip("\n")))
    tgt_batch.append(text_transform[TGT_LANGUAGE](tgt_sample.rstrip("\n")))

  src_batch = pad_sequence(src_batch, padding_value=PAD_IDX)
  tgt_batch = pad_sequence(tgt_batch, padding_value=PAD_IDX)
  return src_batch, tgt_batch

## 訓練のための関数

In [16]:
from torch.utils.data import DataLoader

In [17]:
def train_epoch(model, optimizer):
  model.train()
  losses = 0
  train_iter = Multi30k(split='train', language_pair=(SRC_LANGUAGE, TGT_LANGUAGE))
  train_dataloader = DataLoader(train_iter, batch_size=BATCH_SIZE, collate_fn=collate_fn)
  
  for src, tgt in train_dataloader:
    src = src.to(DEVICE)
    tgt = tgt.to(DEVICE)

    tgt_input = tgt[:-1, :]

    src_mask, tgt_mask, src_padding_mask, tgt_padding_mask = create_mask(src, tgt_input)

    logits = model(src, tgt_input, src_mask, tgt_mask,src_padding_mask, tgt_padding_mask, src_padding_mask)

    optimizer.zero_grad()

    tgt_out = tgt[1:, :]
    loss = loss_fn(logits.reshape(-1, logits.shape[-1]), tgt_out.reshape(-1))
    loss.backward()

    optimizer.step()
    losses += loss.item()

  return losses / len(train_dataloader)

## モデル評価のための関数

In [18]:
def evaluate(model):
  model.eval()
  losses = 0

  val_iter = Multi30k(split='valid', language_pair=(SRC_LANGUAGE, TGT_LANGUAGE))
  val_dataloader = DataLoader(val_iter, batch_size=BATCH_SIZE, collate_fn=collate_fn)

  for src, tgt in val_dataloader:
    src = src.to(DEVICE)
    tgt = tgt.to(DEVICE)

    tgt_input = tgt[:-1, :]

    src_mask, tgt_mask, src_padding_mask, tgt_padding_mask = create_mask(src, tgt_input)

    logits = model(src, tgt_input, src_mask, tgt_mask,src_padding_mask, tgt_padding_mask, src_padding_mask)
    
    tgt_out = tgt[1:, :]
    loss = loss_fn(logits.reshape(-1, logits.shape[-1]), tgt_out.reshape(-1))
    losses += loss.item()

  return losses / len(val_dataloader)

## 実際に学習を実行



In [21]:
from timeit import default_timer as timer
NUM_EPOCHS = 18

for epoch in range(1, NUM_EPOCHS+1):
  start_time = timer()
  train_loss = train_epoch(transformer, optimizer)
  end_time = timer()
  val_loss = evaluate(transformer)
  print((f"Epoch: {epoch}, Train loss: {train_loss:.3f}, Val loss: {val_loss:.3f}, "f"Epoch time = {(end_time - start_time):.3f}s"))

100%|██████████| 46.3k/46.3k [00:00<00:00, 91.5kB/s]


Epoch: 1, Train loss: 5.344, Val loss: 4.135, Epoch time = 79.759s
Epoch: 2, Train loss: 3.793, Val loss: 3.358, Epoch time = 79.612s
Epoch: 3, Train loss: 3.180, Val loss: 2.923, Epoch time = 79.735s
Epoch: 4, Train loss: 2.778, Val loss: 2.639, Epoch time = 79.399s
Epoch: 5, Train loss: 2.489, Val loss: 2.458, Epoch time = 79.609s
Epoch: 6, Train loss: 2.258, Val loss: 2.318, Epoch time = 79.616s
Epoch: 7, Train loss: 2.066, Val loss: 2.217, Epoch time = 79.621s
Epoch: 8, Train loss: 1.904, Val loss: 2.138, Epoch time = 79.766s
Epoch: 9, Train loss: 1.762, Val loss: 2.055, Epoch time = 79.586s
Epoch: 10, Train loss: 1.633, Val loss: 2.004, Epoch time = 79.645s
Epoch: 11, Train loss: 1.522, Val loss: 1.985, Epoch time = 79.504s
Epoch: 12, Train loss: 1.431, Val loss: 1.976, Epoch time = 79.463s
Epoch: 13, Train loss: 1.338, Val loss: 1.953, Epoch time = 79.359s
Epoch: 14, Train loss: 1.252, Val loss: 1.933, Epoch time = 79.646s
Epoch: 15, Train loss: 1.171, Val loss: 1.927, Epoch time

## 学習済みのモデルを使った翻訳

### 貪欲アルゴリズムで翻訳文を生成する関数

In [22]:
def greedy_decode(model, src, src_mask, max_len, start_symbol):
  src = src.to(DEVICE)
  src_mask = src_mask.to(DEVICE)

  memory = model.encode(src, src_mask)
  ys = torch.ones(1, 1).fill_(start_symbol).type(torch.long).to(DEVICE)
  for i in range(max_len-1):
    memory = memory.to(DEVICE)
    tgt_mask = (generate_square_subsequent_mask(ys.size(0)).type(torch.bool)).to(DEVICE)
    out = model.decode(ys, memory, tgt_mask)
    out = out.transpose(0, 1)
    prob = model.generator(out[:, -1])
    _, next_word = torch.max(prob, dim=1) # 貪欲に確率最大の単語を採用
    next_word = next_word.item()

    ys = torch.cat([ys,
                    torch.ones(1, 1).type_as(src.data).fill_(next_word)], dim=0)
    if next_word == EOS_IDX:
      break
  return ys

### 与えられた文を実際に翻訳する関数

In [23]:
def translate(model: torch.nn.Module, src_sentence: str):
  model.eval()
  src = text_transform[SRC_LANGUAGE](src_sentence).view(-1, 1)
  num_tokens = src.shape[0]
  src_mask = (torch.zeros(num_tokens, num_tokens)).type(torch.bool)
  tgt_tokens = greedy_decode(
      model,  src, src_mask, max_len=num_tokens + 5, start_symbol=BOS_IDX).flatten()
  return " ".join(vocab_transform[TGT_LANGUAGE].lookup_tokens(list(tgt_tokens.cpu().numpy()))).replace("<bos>", "").replace("<eos>", "")

### 翻訳の実行

In [24]:
print(translate(transformer, "Eine Gruppe von Menschen steht vor einem Iglu ."))

 A group of people in an office setting . 


## 参考文献
1. Attention is all you need paper.
   https://papers.nips.cc/paper/2017/file/3f5ee243547dee91fbd053c1c4a845aa-Paper.pdf
2. The annotated transformer. https://nlp.seas.harvard.edu/2018/04/03/attention.html#positional-encoding

